In [1]:
pip install holidays

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p38/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import holidays
import time

from Help_Funs import smape, is_holiday

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import tensorflow as tf


s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/train.csv'
file_key_2 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/test.csv'
file_key_3 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/sample_submission.csv'
file_key_4 = 'Tabular-Playground-Series/Tabular-Playground-Sep-2022/TPSSEP22_GDP_data_2017_to_2021.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

bucket_object_4 = bucket.Object(file_key_4)
file_object_4 = bucket_object_4.get()
file_content_stream_4 = file_object_4.get('Body')

## Reading data-files
train = pd.read_csv(file_content_stream_1)
train['date'] = pd.to_datetime(train['date'], format = '%Y-%m-%d')

test = pd.read_csv(file_content_stream_2)
test['date'] = pd.to_datetime(test['date'], format = '%Y-%m-%d')

submission = pd.read_csv(file_content_stream_3)
country_gdp = pd.read_csv(file_content_stream_4)

important_dates = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 124, 125, 126, 127, 140, 141, 167, 168, 169, 170, 171, 173, 174, 175, 176, 177, 178, 179, 
                   180, 181, 203, 230, 231, 232, 233, 234, 282, 289, 290, 307, 308, 309, 310, 311, 312, 313, 317, 318, 319, 320, 360, 361, 362, 363, 364, 365]
    
## Basic feature engineering 
train['weekday'] = train['date'].dt.dayofweek
train['month'] = train['date'].dt.month
train['weekend'] = np.where(train['weekday'] >= 5, 1, 0)
train['dayOfMonth'] = train['date'].dt.day
train['dayOfYear'] = train['date'].dt.dayofyear
train['quarter'] = train['date'].dt.quarter
train['year'] = train['date'].dt.year
train['month_sin'] = np.sin(train['month'] * (2 * np.pi / 12))
# train['month_cos'] = np.cos(train['month'] * (2 * np.pi / 12))
train['day_sin'] = np.sin(train['dayOfMonth'] * (2 * np.pi / 12))
# train['day_cos'] = np.cos(train['dayOfMonth'] * (2 * np.pi / 12))
# train['important_dates'] = train['dayOfYear'].apply(lambda x: x if x in important_dates else 0)

test['weekday'] = test['date'].dt.dayofweek
test['month'] = test['date'].dt.month
test['weekend'] = np.where(test['weekday'] >= 5, 1, 0)
test['dayOfMonth'] = test['date'].dt.day
test['dayOfYear'] = test['date'].dt.dayofyear
test['quarter'] = test['date'].dt.quarter
test['year'] = test['date'].dt.year
test['month_sin'] = np.sin(test['month'] * (2 * np.pi / 12))
# test['month_cos'] = np.cos(test['month'] * (2 * np.pi / 12))
test['day_sin'] = np.sin(test['dayOfMonth'] * (2 * np.pi / 12))
# test['day_cos'] = np.cos(test['dayOfMonth'] * (2 * np.pi / 12))
# test['important_dates'] = test['dayOfYear'].apply(lambda x: x if x in important_dates else 0)

## Appending GDP
train = pd.merge(train, country_gdp, on = ['country', 'year'], how = 'left')
train = train.drop(columns = ['year'], axis = 1)

test = pd.merge(test, country_gdp, on = ['country', 'year'], how = 'left')
test = test.drop(columns = ['year'], axis = 1)

## Appending holidays
data_holidays = is_holiday(train, test)
train = data_holidays[0]
test = data_holidays[1]

train = train.drop(columns = ['mother_day', 'black_friday_cyber_monday'], axis = 1)
test = test.drop(columns = ['mother_day', 'black_friday_cyber_monday'], axis = 1)

/home/ec2-user/SageMaker/Analytics_Data_Science/Tabular-Playground-Series/Tabular-Playground-Sep-2022/Help_Funs.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_temp['is_holiday'][j] = np.where(train_temp['date'][j] in holiday_to_use, 1, 0)
/home/ec2-user/SageMaker/Analytics_Data_Science/Tabular-Playground-Series/Tabular-Playground-Sep-2022/Help_Funs.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_temp['black_friday_cyber_monday'][j] = 0
/home/ec2-user/SageMaker/Analytics_Data_Science/Tabular-Playground-Series/Tabular-Playground-Sep-2022/Help_Funs.py:116: SettingWithCopyWarning: 
A value is trying 

In [7]:
all_data = pd.concat([train, test], axis = 0)

le = LabelEncoder()
cols = ['country', 'store', 'product']
for col in cols:
    le = LabelEncoder()
    all_data[col] = le.fit_transform(all_data[col])

scaler = MinMaxScaler()    

all_data = all_data.drop(['date', 'row_id'], axis = 1)
train = all_data.iloc[:70128,:]
test = all_data.iloc[70128:,:].drop(['num_sold'], axis = 1)
test = pd.DataFrame(scaler.fit_transform(test), columns = test.columns)

X = train.drop(['num_sold'], axis = 1)
Y = train['num_sold']

In [8]:
X.head()

,country,store,product,weekday,month,weekend,dayOfMonth,dayOfYear,quarter,month_sin,day_sin,GDP,is_holiday,holiday_season
0,0,0,0,6,1,1,1,1,1,0.5,0.5,0.5015,1,0
1,0,0,1,6,1,1,1,1,1,0.5,0.5,0.5015,1,0
2,0,0,2,6,1,1,1,1,1,0.5,0.5,0.5015,1,0
3,0,0,3,6,1,1,1,1,1,0.5,0.5,0.5015,1,0
4,0,1,0,6,1,1,1,1,1,0.5,0.5,0.5015,1,0


In [16]:
t1 = time.time()
kf = KFold(n_splits = 4, shuffle = True, random_state = 888)
score_list_tf = []
test_preds_tf = []
fold = 1

## Defining model 
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(16, input_dim = 14, activation = 'relu'),
        tf.keras.layers.Dense(16, activation = 'relu'),
        tf.keras.layers.Dense(1)
])

model.compile(optimizer = 'adam', loss = 'mean_absolute_error')


for train_index, test_index in kf.split(X, Y):
    
    ## Splitting the data
    X_train , X_val = X.iloc[train_index], X.iloc[test_index]  
    Y_train, Y_val = Y.iloc[train_index], Y.iloc[test_index]    
    
    print("X_train shape is :", X_train.shape, "X_val shape is", X_val.shape)
    
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
    X_val = pd.DataFrame(scaler.fit_transform(X_val), columns = X_val.columns)

    model.fit(X_train, Y_train, verbose = 1, epochs = 30, batch_size = 32, validation_data = (X_val, Y_val))
    result = pd.DataFrame(model.predict(X_val))    
    result.columns = ['pred_num_sold']
    result['pred_num_sold'] = [0 if i <= 0 else i for i in result['pred_num_sold']]
    
    Y_val = pd.DataFrame(Y_val.reset_index(drop = True))
    score = smape(Y_val['num_sold'], result['pred_num_sold'])
    print('Fold ', str(fold), ' result is:', score, '\n')
    score_list_tf.append(score)

    test_preds_tf.append(model.predict(test))
    fold += 1

t2 = time.time()
print('TF model with cross validation take : {:.3f} sn.'.format(t2-t1))

X_train shape is : (52596, 14) X_val shape is (17532, 14)
Epoch 1/30
1644/1644 [==============================] - 3s 1ms/step - loss: 96.1640 - val_loss: 59.5056
Epoch 2/30
1644/1644 [==============================] - 2s 1ms/step - loss: 56.2459 - val_loss: 54.3965
Epoch 3/30
1644/1644 [==============================] - 2s 1ms/step - loss: 54.1243 - val_loss: 53.3755
Epoch 4/30
1644/1644 [==============================] - 2s 1ms/step - loss: 52.9883 - val_loss: 52.0513
Epoch 5/30
1644/1644 [==============================] - 2s 1ms/step - loss: 51.1947 - val_loss: 49.4890
Epoch 6/30
1644/1644 [==============================] - 2s 1ms/step - loss: 47.9236 - val_loss: 45.7274
Epoch 7/30
1644/1644 [==============================] - 2s 1ms/step - loss: 44.7391 - val_loss: 43.0119
Epoch 8/30
1644/1644 [==============================] - 3s 2ms/step - loss: 42.2542 - val_loss: 40.5455
Epoch 9/30
1644/1644 [==============================] - 2s 1ms/step - loss: 39.7800 - val_loss: 38.1053
Epoch 

In [17]:
mean = sum(score_list_tf) / len(score_list_tf)
variance = sum([((x - mean) ** 2) for x in score_list_tf]) / len(score_list_tf)
res = variance ** 0.5
print("Cross validation mean score:", sum(score_list_tf) / len(score_list_tf))
print("Cross validation score's Standart deviation is:", res)

Cross validation mean score: 14.513617728209066
Cross validation score's Standart deviation is: 0.7552736898230944


In [11]:
test_preds_tf = pd.DataFrame(np.concatenate(test_preds_tf, axis = 1))
print(test_preds_tf.shape)

test_preds_tf = round(test_preds_tf.mean(axis = 1))
print(test_preds_tf.head())

(17520, 4)
0    452.0
1    334.0
2    297.0
3    469.0
4    165.0
dtype: float32


In [13]:
submission['num_sold'] = test_preds_tf.astype(int)
submission.to_csv('TF_submission_11.csv', index = False)
submission.head()

,row_id,num_sold
0,70128,452
1,70129,334
2,70130,297
3,70131,469
4,70132,165
